In [1]:
import random
import csv

from psycopg2 import connect, extras

from connectors.postgres_connector import PostgresConnector
from feeds.postgres_feed import PostgresFeed

from mangle_org_name import mangle_org_name


source_host = 'localhost'
source_user = 'postgres'
source_passwd = 'postgres'
source_db = 'defaultdb'
source_port = 5432

training_host = 'localhost'
training_user = 'postgres'
training_passwd = 'postgres'
training_db = 'silobuster_testing'
training_port = 5432

csv_output = 'training_set1_testing_th.csv'

print ('Starting connections...')
source_conn = connect(
        database=source_db,
        user=source_user,
        password=source_passwd,
        host=source_host,
        port=source_port
    )

training_conn = connect(
        database=training_db,
        user=training_user,
        password=training_passwd,
        host=training_host,
        port=training_port
    )

select_qry = "select t1.name, t1.description, t1.url, t3.address_1, t3.address_2, t3.city, t3.region, t3.state_province, t3.postal_code, t3.country, t3.type from organization t1 left join location t2 on t1.id = t2.organization_id left join address t3 on t3.location_id = t2.id"
insert_qry = "INSERT INTO organizations_normalized (name, description, url, address_1, address_2, city, state_province, postal_code, country, type, region) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING id"
insert_dup_qry = "INSERT INTO organizations_normalized (name, description, url, address_1, address_2, city, state_province, postal_code, country, type, region, duplicate_id, duplicate_type, training_set) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

print ("Connected?")
dups = []

# Data keys for mangling
address_keys = []
with open('street_suffixes.csv', 'r') as suffixes:
    csv_reader = csv.reader(suffixes)
    for row in csv_reader:
        row_clean = [suffix for suffix in row if suffix] # removing empty strings
        address_keys.append(row_clean)

state_keys = list()
state_keys.append(['wa', 'wash', 'washington'])

with source_conn.cursor(cursor_factory=extras.RealDictCursor) as source_cur:
    source_cur.execute(select_qry)
    data = source_cur.fetchall()
    print ("Retrieved data...")
    
    dup_address = random.randint(1,3)
    dup_blank_stuff = random.randint(1,2)
    
    for count, row in enumerate(data):
        
        print (f"Insert row count: {count}")

                
        name = row['name'].strip().lower()
        try:
            desc = row['description'].strip().lower()
        except:
            desc = ''
        try:
            url = row['url'].strip().lower()
        except:
            url = ''
        try:
            address_1 = row['address_1'].strip().lower()
        except:
            address_1 = ''
        try:
            address_2 = row['address_2'].strip().lower()
        except:
            address_2 = ''
        try:
            city = row['city'].strip().lower()
        except:
            city = ''
        try:
            region = row['region'].strip().lower()
        except:
            region = ''
        try:
            state = row['state_province'].strip().lower()
        except:
            state = ''
        try:
            postal = row['postal_code'].strip().lower()
        except:
            postal = ''
        try:
            country = row['country'].strip().lower()
        except:
            country = ''
        try:
            type_row = row['type'].strip().lower()
        except:
            type_row = ''
        
        # Write the rows to the normalized table
        with training_conn.cursor() as training_cur:
            training_cur.execute(insert_qry, [
                    name, 
                    desc, 
                    url, 
                    address_1, 
                    address_2, 
                    city, 
                    state, 
                    postal,
                    country, 
                    type_row,
                    region, 

            ])
            insert_id = training_cur.fetchone()[0]
            training_conn.commit()
        
        
        with open(csv_output, 'a') as csvfile:
            w = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
            w.writerow([
                name,
                desc,
                url,
                address_1,
                address_2,
                city,
                state,
                postal,
                country,
                type_row,
                region,
                '',
                '',
                0,
            ])

        #create a duplicate/mangled name
        mangled_name = mangle_org_name(
            name, 
            remove_prob = 0.02,
            replace_prob = 0.02,
            null_prob = 0.02
        )

        duplicate_type = []
        if mangled_name != name:
            duplicate_type.append('mangled_name')

        # initializing mangled fields as the original values - this will allow us to identify if these fields were mangled later on
        # when deciding if we need to insert another "duplicate" row
        mangled_address_1 = address_1
        mangled_desc = desc
        mangled_url = url
        mangled_region = region
        mangled_country = country

        #create a duplicate address
        if dup_address == count:
            dup_address = random.randint(1, 3) + count
            duplicate_type.append('street_dup')
            lst = address_1.split(' ')
            for a in address_keys:
                if lst[len(lst)-1] in a:
                    new_street = a[random.randint(0, len(a)-1)]
                    del lst[len(lst)-1]
                    lst.append(new_street)
                    break
                    
            mangled_address_1 = ' '.join(lst)
            if count // dup_blank_stuff == 0:
                duplicate_type.append('blanked')
                mangled_desc = ''
                mangled_url = ''
                mangled_region = ''
                mangled_country = ''
        
        # if any of the fields were mangled, create a duplicate row in database and in training set CSV file
        if mangled_name != name or mangled_address_1 != address_1 or mangled_desc != desc or mangled_url != url or mangled_region != region or mangled_country != country:  
            dup_row = {
                'name': mangled_name,
                'description': mangled_desc,
                'url': mangled_url,
                'address_1': mangled_address_1,
                'address_2': address_2,
                'city': city,
                'region': mangled_region,
                'state': state,
                'postal': postal,
                'country': mangled_country,
                'duplicate_id': insert_id,
                'duplicate_type': duplicate_type,
                'training_set': 1
            }

            # name, description, url, address_1, address_2, city, state_province, postal_code, country, type, region, duplicate_id, duplicate_type, training_set
            
            with training_conn.cursor() as dup1_cur:
                dup1_cur.execute(insert_dup_qry, [
                        mangled_name,
                        mangled_desc,
                        mangled_url,
                        mangled_address_1,
                        address_2,
                        city,
                        state,
                        postal,
                        mangled_country,
                        type_row,
                        mangled_region,
                        insert_id,
                        '|'.join(duplicate_type),
                        1,
                    ])
                
    

            with open(csv_output, 'a') as csvfile:
                w = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
                w.writerow([
                    mangled_name,
                    mangled_desc,
                    mangled_url,
                    mangled_address_1,
                    address_2,
                    city,
                    state,
                    postal,
                    mangled_country,
                    type_row,
                    mangled_region,
                    insert_id,
                    '|'.join(duplicate_type),
                    1,
                ])
                
 
print ('finished')           
            

Starting connections...
Connected?
Retrieved data...
Insert row count: 0
Insert row count: 1
Insert row count: 2
Insert row count: 3
Insert row count: 4
Insert row count: 5
Insert row count: 6
Insert row count: 7
Insert row count: 8
Insert row count: 9
Insert row count: 10
Insert row count: 11
Insert row count: 12
Insert row count: 13
Insert row count: 14
Insert row count: 15
Insert row count: 16
Insert row count: 17
Insert row count: 18
Insert row count: 19
Insert row count: 20
Insert row count: 21
Insert row count: 22
Insert row count: 23
Insert row count: 24
Insert row count: 25
Insert row count: 26
Insert row count: 27
Insert row count: 28
Insert row count: 29
Insert row count: 30
Insert row count: 31
Insert row count: 32
Insert row count: 33
Insert row count: 34
Insert row count: 35
Insert row count: 36
Insert row count: 37
Insert row count: 38
Insert row count: 39
Insert row count: 40
Insert row count: 41
Insert row count: 42
Insert row count: 43
Insert row count: 44
Insert row c